# Imports and Setups

In [1]:
! pip install transformers==4.10.1

     |████████████████████████████████| 2.8 MB 14.4 MB/s 
     |████████████████████████████████| 596 kB 63.9 MB/s 
     |████████████████████████████████| 895 kB 62.6 MB/s 
     |████████████████████████████████| 3.3 MB 53.1 MB/s 
     |████████████████████████████████| 77 kB 9.4 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [2]:
! pip install nlpaug==1.1.7

     |████████████████████████████████| 405 kB 16.4 MB/s 


In [3]:
! pip install sentencepiece

     |████████████████████████████████| 1.2 MB 14.8 MB/s 


In [4]:
! pip install simpletransformers

     |████████████████████████████████| 249 kB 14.2 MB/s 
     |████████████████████████████████| 1.8 MB 67.8 MB/s 
     |████████████████████████████████| 325 kB 70.1 MB/s 
     |████████████████████████████████| 43 kB 2.8 MB/s 
     |████████████████████████████████| 10.1 MB 53.4 MB/s 
     |████████████████████████████████| 181 kB 71.5 MB/s 
     |████████████████████████████████| 144 kB 52.1 MB/s 
     |████████████████████████████████| 63 kB 1.8 MB/s 
     |████████████████████████████████| 136 kB 55.3 MB/s 
     |████████████████████████████████| 1.1 MB 56.6 MB/s 
     |████████████████████████████████| 212 kB 58.0 MB/s 
     |████████████████████████████████| 127 kB 61.7 MB/s 
     |████████████████████████████████| 271 kB 62.1 MB/s 
     |████████████████████████████████| 94 kB 4.0 MB/s 
     |████████████████████████████████| 144 kB 51.2 MB/s 
     |████████████████████████████████| 76 kB 6.7 MB/s 
     |████████████████████████████████| 4.3 MB 42.2 MB/s 
     |███████████████

Lendo o Arquivo

In [1]:
from google.colab import drive
drive.mount('drive')

Mounted at drive


In [2]:
import os
os.environ['WANDB_DISABLED'] = 'true'

In [3]:
import pandas as pd
from tqdm import tqdm
tqdm.pandas()


In [ ]:
dataset = pd.read_csv('/content/drive/My Drive/Colab Notebooks/semeval_ironic/SemEval2018-T3-train-taskA_emoji.txt',sep = '\t')
dataset.head()

,Tweet index,Label,Tweet text
0,1,1,Sweet United Nations video. Just in time for C...
1,2,1,@mrdahl87 We are rumored to have talked to Erv...
2,3,1,Hey there! Nice to see you Minnesota/ND Winter...
3,4,0,3 episodes left I'm dying over here
4,5,1,I can't breathe! was chosen as the most notabl...


In [4]:
num_gen = 5
perc = 100

In [ ]:
X = dataset['Tweet text']
y = dataset['Label']

In [ ]:
base_cru_1p = dataset.copy()

Gerando Base Percentual

In [ ]:
from sklearn.model_selection import train_test_split
X_resto, X_treino_1p, y_resto, y_treino_1p = train_test_split(X, y, stratify=y, test_size=(perc/100), random_state=42) # Selecionando 1%

In [ ]:
import pandas as pd

data = {'texto': X_treino_1p, 'label': y_treino_1p}
base_cru_1p = pd.DataFrame.from_dict(data)

In [ ]:
import pandas as pd

data = {'texto': X, 'label': y}
base_cru_1p = pd.DataFrame.from_dict(data)

In [ ]:
base_cru_1p.head()

,texto,label
0,Sweet United Nations video. Just in time for C...,1
1,@mrdahl87 We are rumored to have talked to Erv...,1
2,Hey there! Nice to see you Minnesota/ND Winter...,1
3,3 episodes left I'm dying over here,0
4,I can't breathe! was chosen as the most notabl...,1


In [5]:
arq = '/content/drive/My Drive/Colab Notebooks/paper_asoc/perc_' +str(perc) + '/base_emotions_'+str(perc)+'_original.csv'
base_cru_1p = pd.read_csv(arq, encoding='utf-8', sep=';', decimal = ',')

Só rodar quando gerar o original

# Treino Classificador

In [6]:
from simpletransformers.classification import ClassificationModel
import pandas as pd
import logging
import sklearn


In [7]:
train_df = base_cru_1p.copy()
train_df.head()

,texto,label
0,im feeling quite sad and sorry for myself but ...,0
1,i feel like i am still looking at a blank canv...,0
2,i feel like a faithful servant,2
3,i am just feeling cranky and blue,3
4,i can have for a treat or if i am feeling festive,1


In [8]:
train_df.rename(columns= {'texto':'text', 'label': 'labels'}, inplace=True)
train_df.head()

,text,labels
0,im feeling quite sad and sorry for myself but ...,0
1,i feel like i am still looking at a blank canv...,0
2,i feel like a faithful servant,2
3,i am just feeling cranky and blue,3
4,i can have for a treat or if i am feeling festive,1


In [ ]:

# train_df = pd.DataFrame(X_train)
# train_df['target'] = y_train

# eval_df = pd.DataFrame(X_test)
# eval_df['target'] = y_test



# Create a ClassificationModel
classifica = ClassificationModel('bert', 'bert-base-uncased', num_labels=6, args={'num_train_epochs':4,
                                                                          'train_batch_size':32,
                                                                          'max_seq_length':64,
                                                                          #'silent':True,
                                                                        #  "fp16":False,
                                                                          'overwrite_output_dir': True}) 

# Train the model
classifica.train_model(train_df)
# result, model_outputs, wrong_predictions = model.eval_model(eval_df, acc=sklearn.metrics.accuracy_score,f1=f1_multiclass)

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

...

Epoch:   0%|          | 0/4 [00:00<?, ?it/s]

Running Epoch 0 of 4:   0%|          | 0/63 [00:00<?, ?it/s]

Running Epoch 1 of 4:   0%|          | 0/63 [00:00<?, ?it/s]

Running Epoch 2 of 4:   0%|          | 0/63 [00:00<?, ?it/s]

Running Epoch 3 of 4:   0%|          | 0/63 [00:00<?, ?it/s]

In [ ]:
test = classifica.predict(["@Funny_Truth: Substitute For Chicken Nuggets h..."])[1]
test

In [ ]:
import numpy as np
mx = np.max(test, axis=-1, keepdims=True)
mx
# array([[1002],
#        [   4]])

# array([[-1,  0],
#        [-1,  0]])
numerator = np.exp(test - mx)
denominator = np.sum(numerator, axis=-1, keepdims=True)
denominator
# array([[ 1.36787944],
#        [ 1.36787944]])
numerator/denominator

In [ ]:
import re
import json
# from sklearn.model_selection import train_test_split



def build_text_files(data_json, dest_path):
    f = open(dest_path, 'w')
    data = ''
    for texts in data_json['texto']:
        summary = str(texts).strip()
        summary = re.sub(r"\s", " ", summary)
        data += summary + "\n"
    f.write(data)




build_text_files(base_cru_1p,'train_dataset.txt')
# build_text_files(test,'test_dataset.txt')

# print("Train dataset length: "+str(len(texto)))
# print("Test dataset length: "+ str(len(test)))


In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("gpt2")

Downloading:   0%|          | 0.00/665 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [ ]:
from transformers import TextDataset,DataCollatorForLanguageModeling

train_path = 'train_dataset.txt'

def load_dataset(train_path,tokenizer):
    train_dataset = TextDataset(
          tokenizer=tokenizer,
          file_path=train_path,
          block_size=128)
     
    
    data_collator = DataCollatorForLanguageModeling(
        tokenizer=tokenizer, mlm=False,
    )
    return train_dataset,data_collator

train_dataset,data_collator = load_dataset(train_path,tokenizer)

/usr/local/lib/python3.7/dist-packages/transformers/data/datasets/language_modeling.py:58: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/master/examples/pytorch/language-modeling/run_mlm.py
  FutureWarning,
Token indices sequence length is longer than the specified maximum sequence length for this model (41307 > 1024). Running this sequence through the model will result in indexing errors


In [ ]:
from transformers import Trainer, TrainingArguments,AutoModelWithLMHead

model = AutoModelWithLMHead.from_pretrained("gpt2")


training_args = TrainingArguments(
    output_dir="./gpt2-model-10p", #The output directory
    overwrite_output_dir=True, #overwrite the content of the output directory
    num_train_epochs=30, # number of training epochs
    per_device_train_batch_size=32, # batch size for training
    per_device_eval_batch_size=64,  # batch size for evaluation
    # eval_steps = 400, # Number of update steps between two evaluations.
    # save_steps=800, # after # steps model is saved 
    # warmup_steps=500,# number of warmup steps for learning rate scheduler
    # prediction_loss_only=True,
    )


trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    # eval_dataset=test_dataset,
)

In [ ]:
trainer.train()

In [ ]:
trainer.save_model()

In [ ]:
from transformers import pipeline

tweet = pipeline('text-generation',model='./gpt2-model-10p', tokenizer='gpt2', device = 0)


In [ ]:
tweet('This is', num_return_sequences = 5, max_length = 20)

In [ ]:
def generate_lambada(texto, token, gerador, classe, classificador, palavras, limiar, num_text, device, max_len, limite):
  texto_curto = token.decode(token.encode(texto, return_tensors="pt").to(device)[0, :palavras])
  res = []
  i=0

  while len(res) < num_text and i < limite:
    gerado = gerador(texto_curto, max_length = max_len)[0]

    # print(gerado['generated_text'])

    predito = classificador.predict([gerado['generated_text']])[1]

    mx = np.max(predito, axis=-1, keepdims=True)
    numerator = np.exp(predito - mx)
    denominator = np.sum(numerator, axis=-1, keepdims=True)

    soft = numerator/denominator

    prob = soft[0][classe]

    if prob >= limiar :
      res.append(gerado['generated_text'])
    i+=1
    
  return res




In [ ]:
generate_lambada( 'This is happy day, I hate it soo much', tokenizer, tweet, 0, classifica, 3, 0.8, 3, 'cuda', 25,20)

Using pad_token, but it is not set yet.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Using pad_token, but it is not set yet.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Using pad_token, but it is not set yet.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Using pad_token, but it is not set yet.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


...

Using pad_token, but it is not set yet.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Using pad_token, but it is not set yet.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


[]

# Gera LAMBADA

In [ ]:
repeticoes = 1

In [ ]:
exp_lambada = []

for i in range(repeticoes):
  base_cru_1p_lambada = base_cru_1p.copy()
  base_cru_1p_lambada['parafraseador'] = base_cru_1p_lambada.progress_apply(lambda x:generate_lambada( x.texto, tokenizer, tweet, x.label, classifica, 5, 0.3, 5, 'cuda', 25, 7), axis=1)
  base_cru_1p_lambada = base_cru_1p_lambada.explode('parafraseador').reset_index(drop=True)
  exp_lambada.append(base_cru_1p_lambada)

  arquivo = '/content/drive/My Drive/Colab Notebooks/paper_asoc/perc_' +str(perc) + '/base_emotions_'+str(perc)+'_lambada'+str(i+1)+'.csv'
  exp_lambada[i].to_csv(arquivo, encoding='utf-8', index=False, sep=';', decimal = ',')
